<a href="https://colab.research.google.com/github/BokyungChoi/2019_Fall_Recommender_System/blob/master/191102_Cotent_based_filtering_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Data
데이터셋 설명: 
- shared_articles.csv 
1. CONTENT SHARED: The article was shared in the platform and is available for users.
2. CONTENT REMOVED: The article was removed from the platform and not available for further recommendation.
* users_interactions.csv
1. VIEW: The user has opened the article.
2. LIKE: The user has liked the article.
3. COMMENT CREATED: The user created a comment in the article.
FOLLOW: The user chose to be notified on any new comment in the article.
4. BOOKMARK: The user has bookmarked the article for easy return in the future.

In [0]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
articles_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/articles-sharing-reading-from-cit-deskdrop/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en


In [0]:
interactions_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/articles-sharing-reading-from-cit-deskdrop/users_interactions.csv')
interactions_df.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


### Data preprocessing and Embedding

In [0]:
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0


In [0]:
# 가중치 설정해 넣는 것
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,   
} 

In [0]:
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])

> naive 하게 인터렉션 5 이하는 제거, 콜드 스타트 방지

In [0]:
# cold start prevention
# keeping in dataset only users with at least 5 interactions (단순 컷)
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()

print('# users: %d' % len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


In [0]:
users_interactions_count_df.head(5)

personId
-9223121837663643404    43
-9212075797126931087     5
-9207251133131336884     7
-9199575329909162940    11
-9196668942822132778     7
dtype: int64

In [0]:
users_with_enough_interactions_df.head(5)

,personId
0,-9223121837663643404
1,-9212075797126931087
2,-9207251133131336884
3,-9199575329909162940
4,-9196668942822132778


In [0]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868


> 여러가지 interaction 지표를 가중치를 사용해 eventstrength 라는 지표로 집약

In [0]:
# aggregate all interactions into one 'interaction' with 부여한 가중치
def smooth_user_preference(x):
    return math.log(1+x, 2)
    # math.log 함수는 (a, Base) =  log(a)/log(Base)
    
# https://www.geeksforgeeks.org/log-functions-python/ 
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()

print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39106


,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
5,-9223121837663643404,-7331393944609614247,1.000000
6,-9223121837663643404,-6872546942144599345,1.000000
7,-9223121837663643404,-6728844082024523434,1.000000
8,-9223121837663643404,-6590819806697898649,1.000000
9,-9223121837663643404,-6558712014192834002,1.584963


### Preparing Evaluation metrics

In [0]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 31284
# interactions on Test set: 7822


In [0]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [0]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

> 향후 모델링 하게 된 것을 평가하는 evaluation metrics & printer class

In [0]:
# Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

### Naive Recommender Modeling

In [0]:
# Popularity Model
# Common 하고, 그냥 단순히 이전에 유저가 consume 하지 않은 items을 추천해줌
# baseline 으로 사용
# To leverage long-tail items to the users with very specific interests 
# 다른 모델을 활용해야 함.

In [0]:
# event strength의 합계가 높을수록 popularity 가 높다
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,contentId,eventStrength
0,-4029704725707465084,307.733799
1,-6783772548752091658,233.762157
2,-133139342397538859,228.024567
3,-8208801367848627943,197.107608
4,-6843047699859121724,193.825208
5,8224860111193157980,189.044680
6,-2358756719610361882,183.110951
7,2581138407738454418,180.282876
8,7507067965574797372,179.094002
9,1469580151036142903,170.548969


In [0]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]
        return recommendations_df
    

In [0]:
popularity_model = PopularityRecommender(item_popularity_df, articles_df)

In [0]:
model_evaluator.evaluate_model(popularity_model)

1139 users processed


({'modelName': 'Popularity',
  'recall@10': 0.37292252620813093,
  'recall@5': 0.2417540271030427},
       hits@5_count  hits@10_count  ...  recall@10           _person_id
 76              28             50  ...   0.260417  3609194402293569455
 17              12             25  ...   0.186567 -2626634673110551643
 16              13             23  ...   0.176923 -1032019229384696495
 10               5              9  ...   0.076923 -1443636648652872475
 82              25             40  ...   0.454545 -2979881261169775358
 ...            ...            ...  ...        ...                  ...
 872              0              0  ...   0.000000 -3946214281191348339
 869              0              0  ...   0.000000  9165571805999894845
 867              0              0  ...   0.000000  5287342266016283959
 865              0              0  ...   0.000000 -2762344524254945371
 1139             0              0  ...   0.000000  1306495113310433590
 
 [1140 rows x 6 columns])

In [0]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
1139 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.2417540271030427, 'recall@10': 0.37292252620813093}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,28,50,192,0.145833,0.260417,3609194402293569455
17,12,25,134,0.089552,0.186567,-2626634673110551643
16,13,23,130,0.100000,0.176923,-1032019229384696495
10,5,9,117,0.042735,0.076923,-1443636648652872475
82,25,40,88,0.284091,0.454545,-2979881261169775358
161,12,18,80,0.150000,0.225000,-3596626804281480007
65,20,33,73,0.273973,0.452055,1116121227607581999
81,17,23,69,0.246377,0.333333,692689608292948411
106,14,18,69,0.202899,0.260870,-9016528795238256703
52,21,28,68,0.308824,0.411765,3636910968448833585


### Content based filtering modeling

In [0]:
# Content-based filtering model
# TF-IDF 라는 테크닉을 사용하는데, 비정제된 텍스트를 벡터로 정형화해주는 기술
# TF, IDF 란? 단어 빈도 - 역 문서 빈도
# 주로 문서의 유사도, 검색 결과의 중요도, 단어들의 중요도 작업에 사용
# 그래서 지금 정형화되지 않은 영어 주소들을 stopwords 이용해 정제
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

> 전처리, 자연어 처리 

In [0]:
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)
# https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Xb0syuczbOQ


In [0]:
#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['title'] + "" + articles_df['text'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<3047x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 638928 stored elements in Compressed Sparse Row format>

In [0]:
articles_df['title']
articles_df['text']

1       All of this work is still very early. The firs...
2       The alarm clock wakes me at 8:00 with stream o...
3       We're excited to share the Google Data Center ...
4       The Aite Group projects the blockchain market ...
5       One of the largest and oldest organizations fo...
                              ...                        
3117    A Liga Ventures, aceleradora de startups espec...
3118    Amazon has launched Chime, a video conferencin...
3119    February 9, 2017 - We begin each year with a l...
3120    At JPMorgan Chase & Co., a learning machine is...
3121    The Acquia Partner Awards Program is comprised...
Name: text, Length: 3047, dtype: object

> 모델링

In [0]:
# 유저 프로파일 모델링
# 아이템 프로파일부터 생성
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

# 유저 프로파일 생성
# 이 부분이 핵심함수, 단일 프로파일 생성
def build_users_profile(person_id, interactions_indexed_df):
    # person_id 로 자른다
    interactions_person_df = interactions_indexed_df.loc[person_id]
    # 그 유저가 사용한 아이템의 프로파일을 불러온다 
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    # eventstrength 가져와서 reshaping
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    # Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(
        user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    # normalizing 해서 마무리
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

# 복수 프로파일 생성, 위 함수 당겨다 씀
def build_users_profiles(): 
    interactions_indexed_df = interactions_full_df[interactions_full_df['contentId'] \
                                                   .isin(articles_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [0]:
user_profiles = build_users_profiles()
# user_profiles 는 딕셔너리 형태로, 그럼 어떤 키와 밸유를 가진지 보자
print(user_profiles.keys())

dict_keys([-9223121837663643404, -9212075797126931087, -9207251133131336884, -9199575329909162940, -9196668942822132778, -9188188261933657343, -9172914609055320039, -9156344805277471150, -9120685872592674274, -9109785559521267180, -9063420486253202900, -9060214117327732109, -9047547311469006438, -9016528795238256703, -9009798162809551896, -9001583565812478106, -8994220765455693336, -8909668725653743114, -8891033171626175843, -8860671864164757449, -8854674432071487111, -8853658195208337106, -8845298781299428018, -8830250090736356260, -8823950498314351783, -8802075878443651241, -8784674845716296727, -8781635134606732409, -8781306637602263252, -8763398617720485024, -8738496712327699923, -8719462623048086192, -8704807962619440953, -8699750646678621887, -8694104221113176052, -8686631410634491662, -8674958742744576254, -8672331451814079632, -8670749047273764903, -8652741825481604192, -8625026691457884033, -8620763856232712186, -8607239111818252463, -8606737560479590536, -8606085472606356565,

In [0]:
myprofile = user_profiles[-9223121837663643404]
print(myprofile.shape)

(1, 5000)


In [0]:
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        myprofile.flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

,token,relevance
0,google,0.264311
1,android,0.134612
2,teste,0.117203
3,code,0.113804
4,cloud,0.105702
5,espresso,0.104444
6,app,0.097841
7,apple,0.096864
8,digital,0.093107
9,candidate,0.088624


In [0]:
# CF modeling
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df

    # 모델 이름 프린트용   
    def get_model_name(self):
        return self.MODEL_NAME

    # 유저 프로파일 기반으로 아이템 추천해줌   
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        # 코사인 유사도 함수로 유사도 벡터 생성
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items : 자료형 전처리 방식 아주 굿
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items

    # 추천 결과 아웃풋
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted (이미 산 제품 제외하여 리스트 생성)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        # 생성한 리스트를 컬럼 이름 지정하여 데이터프레임화
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        # 혹시 item 이 없다고 뜰 경우 에러 핸들링, 추천 결과 아웃풋 튜닝
        # verbose의 구체 의미는 부팅시 상세정보를 보여줌으로써, 어느부분에서 어떤 지연이 있는지 확인가능
        if verbose = True:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

In [0]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
1139 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.41459984658655075, 'recall@10': 0.5241626182562005}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,16,26,192,0.083333,0.135417,3609194402293569455
17,21,35,134,0.156716,0.261194,-2626634673110551643
16,22,34,130,0.169231,0.261538,-1032019229384696495
10,34,54,117,0.290598,0.461538,-1443636648652872475
82,8,15,88,0.090909,0.170455,-2979881261169775358
161,14,23,80,0.175000,0.287500,-3596626804281480007
65,10,15,73,0.136986,0.205479,1116121227607581999
81,11,20,69,0.159420,0.289855,692689608292948411
106,5,10,69,0.072464,0.144928,-9016528795238256703
52,4,11,68,0.058824,0.161765,3636910968448833585


**메트릭 이해**
- hitNcount: Verifying if the current interacted item is among the Top-N recommended items
- Recall: the rate of the interacted items that are ranked among the Top-N recommended items, 이 리콜이 최종적인 추천 결과에 대한 지표  
        #when mixed with a set of non-relevant items


In [0]:
cb_detailed_results_df.sort_values(by='recall@5',ascending=False).head(10)

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
59,4,4,4,1.0,1.0,-7566898119905906259
484,3,3,3,1.0,1.0,-2233054196721773008
366,3,3,3,1.0,1.0,-663224541983667655
488,3,3,3,1.0,1.0,3117868596680657524
258,3,3,3,1.0,1.0,8437083040455072898
306,3,3,3,1.0,1.0,2891568537595470094
150,3,3,3,1.0,1.0,2597011162480707820
333,3,3,3,1.0,1.0,7316935598591113280
99,3,3,3,1.0,1.0,7004923583685899415
575,3,3,3,1.0,1.0,8892482595912468268


In [0]:
cb_detailed_results_df.describe()

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
count,1140.000000,1140.000000,1140.000000,1140.000000,1140.000000,1.140000e+03
mean,2.844737,3.596491,6.861404,0.552488,0.622376,1.008195e+17
std,2.734605,4.011221,12.908814,0.396041,0.401584,5.470049e+18
min,0.000000,0.000000,1.000000,0.000000,0.000000,-9.223122e+18
25%,1.000000,2.000000,2.000000,0.142857,0.261452,-4.751794e+18
50%,2.000000,3.000000,3.000000,0.600000,0.750000,3.306900e+17
75%,4.000000,5.000000,7.000000,1.000000,1.000000,4.795038e+18
max,34.000000,54.000000,192.000000,1.000000,1.000000,9.210531e+18
